# Clase Práctica 1: Búsqueda
---

# Problem and Node

In [88]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)

    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)


class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost


failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.


def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)


def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None):
        return []
    return path_states(node.parent) + [node.state]

# Queues

In [89]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x):
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)

    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]

    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

# Search Algorithms

Con best-first search al variar la función *f(n)* se tienen diferentes algoritmos de búsqueda.

In [90]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))


def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)



In [91]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result


def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result


def depth_first_recursive_search(problem, node=None):
    if node is None:
        node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    elif is_cycle(node):
        return failure
    else:
        for child in expand(problem, node):
            result = depth_first_recursive_search(problem, child)
            if result:
                return result
        return failure

# Bidirectional Best-First Search

In [92]:
def bidirectional_best_first_search(problem_f, f_f, problem_b, f_b, terminated):
    node_f = Node(problem_f.initial)
    node_b = Node(problem_f.goal)
    frontier_f, reached_f = PriorityQueue([node_f], key=f_f), {node_f.state: node_f}
    frontier_b, reached_b = PriorityQueue([node_b], key=f_b), {node_b.state: node_b}
    solution = failure
    while frontier_f and frontier_b and not terminated(solution, frontier_f, frontier_b):
        def S1(node, f):
            return str(int(f(node))) + ' ' + str(path_states(node))
        print('Bi:', S1(frontier_f.top(), f_f), S1(frontier_b.top(), f_b))
        if f_f(frontier_f.top()) < f_b(frontier_b.top()):
            solution = proceed('f', problem_f, frontier_f, reached_f, reached_b, solution)
        else:
            solution = proceed('b', problem_b, frontier_b, reached_b, reached_f, solution)
    return solution

def inverse_problem(problem):
    if isinstance(problem, CountCalls):
        return CountCalls(inverse_problem(problem._object))
    else:
        inv = copy.copy(problem)
        inv.initial, inv.goal = inv.goal, inv.initial
        return inv

In [93]:
def bidirectional_uniform_cost_search(problem_f):
    def terminated(solution, frontier_f, frontier_b):
        n_f, n_b = frontier_f.top(), frontier_b.top()
        return g(n_f) + g(n_b) > g(solution)
    return bidirectional_best_first_search(problem_f, g, inverse_problem(problem_f), g, terminated)

def bidirectional_astar_search(problem_f):
    def terminated(solution, frontier_f, frontier_b):
        nf, nb = frontier_f.top(), frontier_b.top()
        return g(nf) + g(nb) > g(solution)
    problem_f = inverse_problem(problem_f)
    return bidirectional_best_first_search(problem_f, lambda n: g(n) + problem_f.h(n),
                                           problem_b, lambda n: g(n) + problem_b.h(n),
                                           terminated)


def proceed(direction, problem, frontier, reached, reached2, solution):
    node = frontier.pop()
    for child in expand(problem, node):
        s = child.state
        print('proceed', direction, S(child))
        if s not in reached or child.path_cost < reached[s].path_cost:
            frontier.add(child)
            reached[s] = child
            if s in reached2: # Frontiers collide; solution found
                solution2 = (join_nodes(child, reached2[s]) if direction == 'f' else
                             join_nodes(reached2[s], child))
                #print('solution', path_states(solution2), solution2.path_cost,
                # path_states(child), path_states(reached2[s]))
                if solution2.path_cost < solution.path_cost:
                    solution = solution2
    return solution

S = path_states

#A-S-R + B-P-R => A-S-R-P + B-P
def join_nodes(nf, nb):
    """Join the reverse of the backward node nb to the forward node nf."""
    #print('join', S(nf), S(nb))
    join = nf
    while nb.parent is not None:
        cost = join.path_cost + nb.path_cost - nb.parent.path_cost
        join = Node(nb.parent.state, join, nb.action, cost)
        nb = nb.parent
        #print('  now join', S(join), 'with nb', S(nb), 'parent', S(nb.parent))
    return join



# Problemas

# Water Pouring Problems

![](images/water22.png)

En un problema de verter agua se da una colección de jarras, cada una de las cuales tiene un tamaño (capacidad) en litros y un nivel actual de agua (en litros). El objetivo es medir un cierto nivel de agua; puede aparecer en cualquiera de las jarras. Un estado está representado por una tupla de niveles de agua actuales, y las acciones disponibles son:
- `(Fill, i)`: llena la jarra `i` hasta el tope (de un grifo con agua ilimitada).
- `(Dump, i)`: vuelca toda el agua de la `i`ésima jarra.
- `(Pour, i, j)`: vierte agua de la jarra `i` en la jarra `j` hasta que la jarra `i` esté vacía o la jarra `j` esté llena, lo que ocurra primero.

In [94]:
class PourProblem(Problem):
    """Problem about pouring water between jugs to achieve some water level.
    Each state is a tuples of water levels. In the initialization, also provide a tuple of
    jug sizes, e.g. PourProblem(initial=(0, 0), goal=4, sizes=(5, 3)),
    which means two jugs of sizes 5 and 3, initially both empty, with the goal
    of getting a level of 4 in either jug."""

    def actions(self, state):
        """The actions executable in this state."""
        jugs = range(len(state))
        return ([('Fill', i)    for i in jugs if state[i] < self.sizes[i]] +
                [('Dump', i)    for i in jugs if state[i]] +
                [('Pour', i, j) for i in jugs if state[i] for j in jugs if i != j])

    def result(self, state, action):
        """The state that results from executing this action in this state."""
        result = list(state)
        act, i, *_ = action
        if act == 'Fill':   # Fill i to capacity
            result[i] = self.sizes[i]
        elif act == 'Dump': # Empty i
            result[i] = 0
        elif act == 'Pour': # Pour from i into j
            j = action[2]
            amount = min(state[i], self.sizes[j] - state[j])
            result[i] -= amount
            result[j] += amount
        return tuple(result)

    def is_goal(self, state):
        """True if the goal level is in any one of the jugs."""
        return self.goal in state

En un `GreenPourProblem`, los estados y las acciones son los mismos, pero en lugar de que todas las acciones cuesten 1, en estos problemas el costo de una acción es la cantidad de agua que sale del grifo. (Existe el problema de que las acciones que no son *Fill* tienen un costo de 0, lo que en general puede llevar a soluciones indefinidamente largas, pero en este problema hay un número finito de estados, por lo que estamos bien).

In [95]:
class GreenPourProblem(PourProblem):
    """A PourProblem in which the cost is the amount of water used."""
    def action_cost(self, s, action, s1):
        "The cost is the amount of water used."
        act, i, *_ = action
        return self.sizes[i] - s[i] if act == 'Fill' else 0

In [96]:
# Some specific PourProblems

p1 = PourProblem((1, 1, 1), 13, sizes=(2, 16, 32))
p2 = PourProblem((0, 0, 0), 21, sizes=(8, 11, 31))
p3 = PourProblem((0, 0),     8, sizes=(7,9))
p4 = PourProblem((0, 0, 0), 21, sizes=(8, 11, 31))
p5 = PourProblem((0, 0),     4, sizes=(3, 5))

g1 = GreenPourProblem((1, 1, 1), 13, sizes=(2, 16, 32))
g2 = GreenPourProblem((0, 0, 0), 21, sizes=(8, 11, 31))
g3 = GreenPourProblem((0, 0),     8, sizes=(7,9))
g4 = GreenPourProblem((0, 0, 0), 21, sizes=(8, 11, 31))
g5 = GreenPourProblem((0, 0),     4, sizes=(3, 5))

In [97]:
soln = breadth_first_search(p1)
path_actions(soln), path_states(soln)

([('Fill', 1), ('Pour', 1, 0), ('Dump', 0), ('Pour', 1, 0)],
 [(1, 1, 1), (1, 16, 1), (2, 15, 1), (0, 15, 1), (2, 13, 1)])

# 8 Puzzle Problems

![](images/puz3.png)

Un puzzle de piezas deslizantes donde puedes intercambiar el espacio en blanco con una pieza adyacente, tratando de alcanzar una configuración ordenada. Las celdas están numeradas del 0 al 8, comenzando en la parte superior izquierda y siguiendo fila por fila de izquierda a derecha. Las piezas están numeradas del 1 al 8, siendo el 0 el espacio en blanco. Una acción es el número de índice de celda que se intercambiará con el espacio en blanco (*no* el número real que se intercambiará sino el índice en el estado). Entonces, el diagrama de arriba a la izquierda representa el estado `(5, 2, 7, 8, 4, 0, 1, 3, 6)`, y la acción es `8`, porque la celda número 8 (la novena o última celda, el `6` en la parte inferior derecha) se intercambia con el espacio en blanco.

Hay dos conjuntos separados de estados a los que no se puede llegar uno desde el otro. Un conjunto tiene un número par de "inversiones"; el otro tiene un número impar. Una inversión es cuando una pieza en el estado es más grande que una pieza que le sigue.

In [98]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal

    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]

    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)

    def h1(self, node):
        """The misplaced tiles heuristic."""
        return hamming_distance(node.state, self.goal)

    def h2(self, node):
        """The Manhattan heuristic."""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                   for (s, g) in zip(node.state, self.goal) if s != 0)

    def h(self, node): return self.h2(node)


def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b for a, b in zip(A, B))


def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))


def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')

class Board(defaultdict):
    empty = '.'
    off = '#'
    def __init__(self, board=None, width=8, height=8, to_move=None, **kwds):
        if board is not None:
            self.update(board)
            self.width, self.height = (board.width, board.height)
        else:
            self.width, self.height = (width, height)
        self.to_move = to_move

    def __missing__(self, key):
        x, y = key
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return self.off
        else:
            return self.empty

    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(row(y) for y in range(self.height))

    def __hash__(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

In [99]:
# Some specific EightPuzzle problems

e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))

In [100]:
# Solve an 8 puzzle problem and print out each state

for s in path_states(astar_search(e1)):
    print(board8(s))

1 4 2
_ 7 5
3 6 8

1 4 2
3 7 5
_ 6 8

1 4 2
3 7 5
6 _ 8

1 4 2
3 _ 5
6 7 8

1 _ 2
3 4 5
6 7 8

_ 1 2
3 4 5
6 7 8



# Jumping Frogs Puzzle

En este puzzle (que también se puede jugar como un juego de dos jugadores), el estado inicial es una línea de cuadrados, con N piezas de un tipo a la izquierda, luego un cuadrado vacío, luego N piezas de otro tipo a la derecha. . El siguiente diagrama usa 2 sapos azules y 2 ranas rojas; representaremos esto como la cadena `'LL.RR'`. El objetivo es intercambiar las piezas, llegando a `'RR.LL'`. Una pieza `'L'` se mueve de izquierda a derecha, ya sea deslizándose un espacio hacia adelante a un espacio vacío, o dos espacios hacia adelante si ese espacio está vacío y si hay una `'R'` en el medio para saltar. Las piezas `'R'` se mueven de derecha a izquierda de forma análoga. Una acción será un par `(i, j)` que significa intercambiar las piezas en esos índices. El conjunto de acciones para la posición N = 2 a continuación es `{(1, 2), (3, 2)}`, lo que significa que el sapo azul en la posición 1 o la rana roja en la posición 3 pueden intercambiar lugares con el espacio en blanco en posición 2.

![](images/ToadsAndFrogs.png)

In [101]:
class JumpingPuzzle(Problem):
    """Try to exchange L and R by moving one ahead or hopping two ahead."""
    def __init__(self, N=2):
        self.initial = N*'L' + '.' + N*'R'
        self.goal = self.initial[::-1]

    def actions(self, state):
        """Find all possible move or hop moves."""
        idxs = range(len(state))
        return ({(i, i + 1) for i in idxs if state[i:i+2] == 'L.'}   # Slide
               |{(i, i + 2) for i in idxs if state[i:i+3] == 'LR.'}  # Hop
               |{(i + 1, i) for i in idxs if state[i:i+2] == '.R'}   # Slide
               |{(i + 2, i) for i in idxs if state[i:i+3] == '.LR'}) # Hop

    def result(self, state, action):
        """An action (i, j) means swap the pieces at positions i and j."""
        i, j = action
        result = list(state)
        result[i], result[j] = state[j], state[i]
        return ''.join(result)

    def h(self, node): return hamming_distance(node.state, self.goal)

In [102]:
JumpingPuzzle(N=2).actions('LL.RR')

{(1, 2), (3, 2)}

In [103]:
j3 = JumpingPuzzle(N=3)
j9 = JumpingPuzzle(N=9)
path_states(astar_search(j3))

['LLL.RRR',
 'LLLR.RR',
 'LL.RLRR',
 'L.LRLRR',
 'LRL.LRR',
 'LRLRL.R',
 'LRLRLR.',
 'LRLR.RL',
 'LR.RLRL',
 '.RLRLRL',
 'R.LRLRL',
 'RRL.LRL',
 'RRLRL.L',
 'RRLR.LL',
 'RR.RLLL',
 'RRR.LLL']

# Pancake Sorting Problems

Dada una pila de pancakes de varios tamaños, ¿puedes ordenarlos en una pila de tamaños decrecientes, del más grande en la parte inferior al más pequeño en la parte superior? Tienes una espátula con la que puedes voltear los `i` pancakes de arriba. Esto se muestra a continuación para `i = 3`; en la parte superior, la espátula agarra los primeros tres pancakes; en la parte inferior los vemos volteados:

![](images/Pancake_sort_operation.png)

¿Cuántas vueltas se necesitarán para ordenar toda la pila? Este es un problema interesante sobre el que Bill Gates ha [escrito](https://people.eecs.berkeley.edu/~christos/papers/Bounds%20For%20Sorting%20By%20Prefix%20Reversal.pdf). Una heurística razonable para este problema es la *gap heuristic*: si observamos panqueques vecinos, por ejemplo, el segundo más pequeño está al lado del tercero más pequeño, está bien; deben permanecer uno al lado del otro. Pero si el 2º más pequeño está al lado del 4º más pequeño, eso es malo: necesitaremos al menos un movimiento para separarlos e insertar el 3º más pequeño entre ellos. La heurística cuenta el número de vecinos que tienen una brecha como esta. En nuestra especificación del problema, los panqueques se clasifican por tamaño: el más pequeño es "1", el segundo más pequeño es "2", y así sucesivamente, y la representación de un estado es una tupla de estas clasificaciones, de arriba hacia abajo. Por lo tanto, el estado objetivo siempre es `(1, 2, ..., `*n*`)` y el estado inicial (superior) en el diagrama anterior es `(2, 1, 4, 6, 3, 5)` .


In [143]:
class PancakeProblem(Problem):
    """A PancakeProblem the goal is always `tuple(range(1, n+1))`, where the
    initial state is a permutation of `range(1, n+1)`. An act is the index `i`
    of the top `i` pancakes that will be flipped."""

    def __init__(self, initial):
        self.initial, self.goal = tuple(initial), tuple(sorted(initial))

    def actions(self, state): # TODO: Your code here!!!
        n=len(state)
        return tuple(range(2,n+1))
    
    def invert(self,state,i):
        answer=[state[e] for e in range(0,i)]
        return list(reversed(answer))
        
    def result(self, state, i): # TODO: Your code here!!!
        state_list=list(state)
        second_part=[state_list[element] for element in range(i,len(state_list))]
        first_part=self.invert(state_list,i)
        answer=first_part+second_part
        return tuple(answer)

    # def h(self, node): return self.gap_h(node)
    def h(self, node): return self.inversion_count_h(node)


    def gap_h(self, node):
        """The Gap Heuristic. Para cada pancake en la posicion i si la diferencia al siguiente pancake es 1 la cuento"""
        answer=0
        for n in range(0,len(node.state)-1):
            if node.state[n+1]-node.state[n] == 1:
                answer=answer+1
        return answer

    def inversion_count_h(self, node):
        """Inversion Count Heuristic."""
        inversion_count = 0
        for i in range(len(node.state) - 1):
            for j in range(i + 1, len(node.state)):
                if node.state[i] > node.state[j]:
                    inversion_count += 1
        return inversion_count

In [144]:
c0 = PancakeProblem((2, 1, 4, 6, 3, 5))
c1 = PancakeProblem((4, 6, 2, 5, 1, 3))
c2 = PancakeProblem((1, 3, 7, 5, 2, 6, 4))
c3 = PancakeProblem((1, 7, 2, 6, 3, 5, 4))
c4 = PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 8))

In [145]:
# Solve a pancake problem using A*
print("c0=PancakeProblem((2, 1, 4, 6, 3, 5))using A*")
sln = astar_search(c0)
path_actions(sln), path_states(astar_search(c0))


c0=PancakeProblem((2, 1, 4, 6, 3, 5))using A*


([4, 6, 5, 4, 3],
 [(2, 1, 4, 6, 3, 5),
  (6, 4, 1, 2, 3, 5),
  (5, 3, 2, 1, 4, 6),
  (4, 1, 2, 3, 5, 6),
  (3, 2, 1, 4, 5, 6),
  (1, 2, 3, 4, 5, 6)])

In [129]:
print("c1 = PancakeProblem((4, 6, 2, 5, 1, 3)),using A*")
sln = astar_search(c1)
path_actions(sln), path_states(astar_search(c1))


c1 = PancakeProblem((4, 6, 2, 5, 1, 3)),using A*


([5, 6, 3, 4, 5, 2, 6],
 [(4, 6, 2, 5, 1, 3),
  (1, 5, 2, 6, 4, 3),
  (3, 4, 6, 2, 5, 1),
  (6, 4, 3, 2, 5, 1),
  (2, 3, 4, 6, 5, 1),
  (5, 6, 4, 3, 2, 1),
  (6, 5, 4, 3, 2, 1),
  (1, 2, 3, 4, 5, 6)])

In [108]:
print("c2 = PancakeProblem((1, 3, 7, 5, 2, 6, 4)),using A*")
sln = astar_search(c2)
path_actions(sln), path_states(astar_search(c2))


c2 = PancakeProblem((1, 3, 7, 5, 2, 6, 4)),using A*


([7, 5, 3, 6, 5, 3, 2, 7],
 [(1, 3, 7, 5, 2, 6, 4),
  (4, 6, 2, 5, 7, 3, 1),
  (7, 5, 2, 6, 4, 3, 1),
  (2, 5, 7, 6, 4, 3, 1),
  (3, 4, 6, 7, 5, 2, 1),
  (5, 7, 6, 4, 3, 2, 1),
  (6, 7, 5, 4, 3, 2, 1),
  (7, 6, 5, 4, 3, 2, 1),
  (1, 2, 3, 4, 5, 6, 7)])

In [109]:
print("c3 = PancakeProblem((1, 7, 2, 6, 3, 5, 4)),using A*")
sln = astar_search(c3)
path_actions(sln), path_states(astar_search(c3))


c3 = PancakeProblem((1, 7, 2, 6, 3, 5, 4)),using A*


([2, 7, 2, 3, 4, 6],
 [(1, 7, 2, 6, 3, 5, 4),
  (7, 1, 2, 6, 3, 5, 4),
  (4, 5, 3, 6, 2, 1, 7),
  (5, 4, 3, 6, 2, 1, 7),
  (3, 4, 5, 6, 2, 1, 7),
  (6, 5, 4, 3, 2, 1, 7),
  (1, 2, 3, 4, 5, 6, 7)])

In [110]:
print("c4 = PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 8)),using A*")
sln = astar_search(c4)
path_actions(sln), path_states(astar_search(c4))

c4 = PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 8)),using A*


([5, 9, 8, 7, 6, 5, 4, 3, 2],
 [(1, 3, 5, 7, 9, 2, 4, 6, 8),
  (9, 7, 5, 3, 1, 2, 4, 6, 8),
  (8, 6, 4, 2, 1, 3, 5, 7, 9),
  (7, 5, 3, 1, 2, 4, 6, 8, 9),
  (6, 4, 2, 1, 3, 5, 7, 8, 9),
  (5, 3, 1, 2, 4, 6, 7, 8, 9),
  (4, 2, 1, 3, 5, 6, 7, 8, 9),
  (3, 1, 2, 4, 5, 6, 7, 8, 9),
  (2, 1, 3, 4, 5, 6, 7, 8, 9),
  (1, 2, 3, 4, 5, 6, 7, 8, 9)])

In [111]:
# Solve a pancake problem BFS
print("c0=PancakeProblem((2, 1, 4, 6, 3, 5))using BFS")
soln = breadth_first_search(c0)
path_actions(soln), path_states(soln)

c0=PancakeProblem((2, 1, 4, 6, 3, 5))using BFS


([4, 6, 5, 4, 3],
 [(2, 1, 4, 6, 3, 5),
  (6, 4, 1, 2, 3, 5),
  (5, 3, 2, 1, 4, 6),
  (4, 1, 2, 3, 5, 6),
  (3, 2, 1, 4, 5, 6),
  (1, 2, 3, 4, 5, 6)])

In [112]:
print("c1 = PancakeProblem((4, 6, 2, 5, 1, 3)),using BFS")
soln = breadth_first_search(c1)
path_actions(soln), path_states(soln)

c1 = PancakeProblem((4, 6, 2, 5, 1, 3)),using BFS


([2, 3, 4, 2, 6, 3, 2],
 [(4, 6, 2, 5, 1, 3),
  (6, 4, 2, 5, 1, 3),
  (2, 4, 6, 5, 1, 3),
  (5, 6, 4, 2, 1, 3),
  (6, 5, 4, 2, 1, 3),
  (3, 1, 2, 4, 5, 6),
  (2, 1, 3, 4, 5, 6),
  (1, 2, 3, 4, 5, 6)])

In [113]:
print("c2 = PancakeProblem((1, 3, 7, 5, 2, 6, 4)),using BFS")
soln = breadth_first_search(c2)
path_actions(soln), path_states(soln)

c2 = PancakeProblem((1, 3, 7, 5, 2, 6, 4)),using BFS


([2, 6, 3, 2, 4, 7, 4, 2],
 [(1, 3, 7, 5, 2, 6, 4),
  (3, 1, 7, 5, 2, 6, 4),
  (6, 2, 5, 7, 1, 3, 4),
  (5, 2, 6, 7, 1, 3, 4),
  (2, 5, 6, 7, 1, 3, 4),
  (7, 6, 5, 2, 1, 3, 4),
  (4, 3, 1, 2, 5, 6, 7),
  (2, 1, 3, 4, 5, 6, 7),
  (1, 2, 3, 4, 5, 6, 7)])

In [114]:
print("c3 = PancakeProblem((1, 7, 2, 6, 3, 5, 4)),using BFS")
soln = breadth_first_search(c3)
path_actions(soln), path_states(soln)

c3 = PancakeProblem((1, 7, 2, 6, 3, 5, 4)),using BFS


([2, 7, 2, 3, 4, 6],
 [(1, 7, 2, 6, 3, 5, 4),
  (7, 1, 2, 6, 3, 5, 4),
  (4, 5, 3, 6, 2, 1, 7),
  (5, 4, 3, 6, 2, 1, 7),
  (3, 4, 5, 6, 2, 1, 7),
  (6, 5, 4, 3, 2, 1, 7),
  (1, 2, 3, 4, 5, 6, 7)])

In [115]:
print("c4 = PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 8)),using BFS")
soln = breadth_first_search(c4)
path_actions(soln), path_states(soln)

c4 = PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 8)),using BFS


([5, 9, 8, 7, 6, 5, 4, 3, 2],
 [(1, 3, 5, 7, 9, 2, 4, 6, 8),
  (9, 7, 5, 3, 1, 2, 4, 6, 8),
  (8, 6, 4, 2, 1, 3, 5, 7, 9),
  (7, 5, 3, 1, 2, 4, 6, 8, 9),
  (6, 4, 2, 1, 3, 5, 7, 8, 9),
  (5, 3, 1, 2, 4, 6, 7, 8, 9),
  (4, 2, 1, 3, 5, 6, 7, 8, 9),
  (3, 1, 2, 4, 5, 6, 7, 8, 9),
  (2, 1, 3, 4, 5, 6, 7, 8, 9),
  (1, 2, 3, 4, 5, 6, 7, 8, 9)])

In [116]:
print("c0=PancakeProblem((2, 1, 4, 6, 3, 5))using greedy_bfs")
soll=greedy_bfs(c0)
path_actions(soll), path_states(soll)


c0=PancakeProblem((2, 1, 4, 6, 3, 5))using greedy_bfs


([4, 6, 5, 4, 3],
 [(2, 1, 4, 6, 3, 5),
  (6, 4, 1, 2, 3, 5),
  (5, 3, 2, 1, 4, 6),
  (4, 1, 2, 3, 5, 6),
  (3, 2, 1, 4, 5, 6),
  (1, 2, 3, 4, 5, 6)])

In [117]:
print("c1 = PancakeProblem((4, 6, 2, 5, 1, 3)),using greedy_bfs")
soll=greedy_bfs(c1)
path_actions(soll), path_states(soll)

c1 = PancakeProblem((4, 6, 2, 5, 1, 3)),using greedy_bfs


([5, 6, 3, 4, 5, 2, 6],
 [(4, 6, 2, 5, 1, 3),
  (1, 5, 2, 6, 4, 3),
  (3, 4, 6, 2, 5, 1),
  (6, 4, 3, 2, 5, 1),
  (2, 3, 4, 6, 5, 1),
  (5, 6, 4, 3, 2, 1),
  (6, 5, 4, 3, 2, 1),
  (1, 2, 3, 4, 5, 6)])

In [118]:
print("c2 = PancakeProblem((1, 3, 7, 5, 2, 6, 4)),using greedy_bfs")
soll=greedy_bfs(c2)
path_actions(soll), path_states(soll)

c2 = PancakeProblem((1, 3, 7, 5, 2, 6, 4)),using greedy_bfs


([7, 5, 3, 6, 5, 3, 2, 7],
 [(1, 3, 7, 5, 2, 6, 4),
  (4, 6, 2, 5, 7, 3, 1),
  (7, 5, 2, 6, 4, 3, 1),
  (2, 5, 7, 6, 4, 3, 1),
  (3, 4, 6, 7, 5, 2, 1),
  (5, 7, 6, 4, 3, 2, 1),
  (6, 7, 5, 4, 3, 2, 1),
  (7, 6, 5, 4, 3, 2, 1),
  (1, 2, 3, 4, 5, 6, 7)])

In [119]:
print("c3 = PancakeProblem((1, 7, 2, 6, 3, 5, 4)),using greedy_bfs")
soll=greedy_bfs(c3)
path_actions(soll), path_states(soll)

c3 = PancakeProblem((1, 7, 2, 6, 3, 5, 4)),using greedy_bfs


([2, 7, 2, 3, 4, 6],
 [(1, 7, 2, 6, 3, 5, 4),
  (7, 1, 2, 6, 3, 5, 4),
  (4, 5, 3, 6, 2, 1, 7),
  (5, 4, 3, 6, 2, 1, 7),
  (3, 4, 5, 6, 2, 1, 7),
  (6, 5, 4, 3, 2, 1, 7),
  (1, 2, 3, 4, 5, 6, 7)])

In [120]:
print("c4 = PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 8)),using greedy_bfs")
soll=greedy_bfs(c4)
path_actions(soll), path_states(soll)

c4 = PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 8)),using greedy_bfs


([5, 9, 8, 7, 6, 5, 4, 3, 2],
 [(1, 3, 5, 7, 9, 2, 4, 6, 8),
  (9, 7, 5, 3, 1, 2, 4, 6, 8),
  (8, 6, 4, 2, 1, 3, 5, 7, 9),
  (7, 5, 3, 1, 2, 4, 6, 8, 9),
  (6, 4, 2, 1, 3, 5, 7, 8, 9),
  (5, 3, 1, 2, 4, 6, 7, 8, 9),
  (4, 2, 1, 3, 5, 6, 7, 8, 9),
  (3, 1, 2, 4, 5, 6, 7, 8, 9),
  (2, 1, 3, 4, 5, 6, 7, 8, 9),
  (1, 2, 3, 4, 5, 6, 7, 8, 9)])

# Análisis de Algoritmos de Búsqueda

Usaremos `CountCalls` para envolver el objeto `Problem` de tal manera que las llamadas a sus métodos se deleguen al problema original, pero cada llamada incrementa un contador. Una vez que hemos resuelto el problema, imprimimos un resumen estadísticas.

In [121]:
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()

    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)


def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts;
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: report_counts(counts, str(p)[:40])
        report_counts(total_counts, 'TOTAL\n')

def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} nodes |{:9,d} goal |{:5.0f} cost |{:8,d} actions | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['actions'], name))

Aquí hay un pequeño informe para uniform-cost search en water pouring problems:

In [122]:
report([uniform_cost_search], [p1, p2, p3, p4, p5])

uniform_cost_search:
      948 nodes |      109 goal |    4 cost |     112 actions | PourProblem((1, 1, 1), 13)
    3,499 nodes |      389 goal |    9 cost |     397 actions | PourProblem((0, 0, 0), 21)
      124 nodes |       30 goal |   14 cost |      43 actions | PourProblem((0, 0), 8)
    3,499 nodes |      389 goal |    9 cost |     397 actions | PourProblem((0, 0, 0), 21)
       52 nodes |       14 goal |    6 cost |      19 actions | PourProblem((0, 0), 4)
    8,122 nodes |      931 goal |   42 cost |     968 actions | TOTAL



In [123]:
report((uniform_cost_search, breadth_first_search),
       (p1, p2, p3, p4, p4, c1, c2, c3))

uniform_cost_search:
      948 nodes |      109 goal |    4 cost |     112 actions | PourProblem((1, 1, 1), 13)
    3,499 nodes |      389 goal |    9 cost |     397 actions | PourProblem((0, 0, 0), 21)
      124 nodes |       30 goal |   14 cost |      43 actions | PourProblem((0, 0), 8)
    3,499 nodes |      389 goal |    9 cost |     397 actions | PourProblem((0, 0, 0), 21)
    3,499 nodes |      389 goal |    9 cost |     397 actions | PourProblem((0, 0, 0), 21)
    3,590 nodes |      719 goal |    7 cost |     725 actions | PancakeProblem((4, 6, 2, 5, 1, 3), (1, 2


   30,204 nodes |    5,035 goal |    8 cost |   5,042 actions | PancakeProblem((1, 3, 7, 5, 2, 6, 4), (1
   22,068 nodes |    3,679 goal |    6 cost |   3,684 actions | PancakeProblem((1, 7, 2, 6, 3, 5, 4), (1
   67,431 nodes |   10,739 goal |   66 cost |  10,797 actions | TOTAL

breadth_first_search:
      596 nodes |      597 goal |    4 cost |      73 actions | PourProblem((1, 1, 1), 13)
    2,618 nodes |    2,619 goal |    9 cost |     302 actions | PourProblem((0, 0, 0), 21)
      120 nodes |      121 goal |   14 cost |      42 actions | PourProblem((0, 0), 8)
    2,618 nodes |    2,619 goal |    9 cost |     302 actions | PourProblem((0, 0, 0), 21)
    2,618 nodes |    2,619 goal |    9 cost |     302 actions | PourProblem((0, 0, 0), 21)
    2,951 nodes |    2,952 goal |    7 cost |     598 actions | PancakeProblem((4, 6, 2, 5, 1, 3), (1, 2
   25,945 nodes |   25,946 goal |    8 cost |   4,333 actions | PancakeProblem((1, 3, 7, 5, 2, 6, 4), (1
    5,975 nodes |    5,976 goal |   

# Comparación de heurísticas

Veamos el eight puzzle problem y comparemos tres heurísticas diferentes: la heurística de Manhattan, la heurística de piezas fuera de lugar menos informativa y la búsqueda desinformada (es decir, *h* = 0) breadth-first search:

In [124]:
def astar_misplaced_tiles(problem): return astar_search(problem, h=problem.h1)

report([breadth_first_search, astar_misplaced_tiles, astar_search],
       [e1, e2, e3, e4, e5])

breadth_first_search:
       81 nodes |       82 goal |    5 cost |      35 actions | EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8),


  160,948 nodes |  160,949 goal |   22 cost |  59,960 actions | EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0),
  218,263 nodes |  218,264 goal |   23 cost |  81,829 actions | EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6),
  418,771 nodes |  418,772 goal |   26 cost | 156,533 actions | EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1),
  448,667 nodes |  448,668 goal |   27 cost | 167,799 actions | EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1),
1,246,730 nodes |1,246,735 goal |  103 cost | 466,156 actions | TOTAL

astar_misplaced_tiles:
       17 nodes |        7 goal |    5 cost |      11 actions | EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8),
   23,407 nodes |    8,726 goal |   22 cost |   8,747 actions | EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0),
   38,632 nodes |   14,433 goal |   23 cost |  14,455 actions | EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6),
  124,324 nodes |   46,553 goal |   26 cost |  46,578 actions | EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1),
  156,111 nodes |   58,475 goal |   27 cost |  58,501 actions | Ei

Vemos que los tres algoritmos obtienen soluciones óptimas, pero cuanto mejor es la heurística, menos nodos se exploran.
En comparación con la búsqueda no informada, la heurística de piezas fuera de lugar explora alrededor de 1/4 de la cantidad de nodos, y la heurística de Manhattan necesita solo el 2 %.

A continuación, podemos ver el valor de la heurística de brecha para pancake sorting problems:

In [141]:
def astar_gap(problem): return astar_search(problem, h = problem.gap_h)
def astar_inversion(problem): return astar_search(problem, h= problem.inversion_count_h)

report([astar_inversion, astar_gap, breadth_first_search], [c1, c2, c3, c4])

astar_inversion:
      755 nodes |      152 goal |    8 cost |     159 actions | PancakeProblem((4, 6, 2, 5, 1, 3), (1, 2
    2,922 nodes |      488 goal |   10 cost |     497 actions | PancakeProblem((1, 3, 7, 5, 2, 6, 4), (1
    2,004 nodes |      335 goal |   11 cost |     345 actions | PancakeProblem((1, 7, 2, 6, 3, 5, 4), (1
  152,504 nodes |   19,064 goal |    9 cost |  19,072 actions | PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 
  158,185 nodes |   20,039 goal |   38 cost |  20,073 actions | TOTAL

astar_gap:
    3,675 nodes |      736 goal |    7 cost |     742 actions | PancakeProblem((4, 6, 2, 5, 1, 3), (1, 2
   31,182 nodes |    5,198 goal |    8 cost |   5,205 actions | PancakeProblem((1, 3, 7, 5, 2, 6, 4), (1
   31,386 nodes |    5,232 goal |    6 cost |   5,237 actions | PancakeProblem((1, 7, 2, 6, 3, 5, 4), (1
3,037,560 nodes |  379,696 goal |    9 cost | 379,704 actions | PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 
3,103,803 nodes |  390,862 goal |   30 cost | 390,888 actions

Necesitamos explorar casi 300 veces más nodos sin la heurística.

# Comparación de algoritmos de búsqueda

In [126]:
report((astar_search, uniform_cost_search,  breadth_first_search, breadth_first_bfs,
        iterative_deepening_search, depth_limited_search, greedy_bfs,
        weighted_astar_search),
       (p1, p2, g1, g2, j3, j9, c1, c2))

astar_search:
      948 nodes |      109 goal |    4 cost |     112 actions | PourProblem((1, 1, 1), 13)
    3,499 nodes |      389 goal |    9 cost |     397 actions | PourProblem((0, 0, 0), 21)
    1,696 nodes |      190 goal |   10 cost |     204 actions | GreenPourProblem((1, 1, 1), 13)


    4,072 nodes |      454 goal |   21 cost |     463 actions | GreenPourProblem((0, 0, 0), 21)
       81 nodes |       71 goal |   15 cost |      85 actions | JumpingPuzzle('LLL.RRR', 'RRR.LLL')
   10,991 nodes |   10,661 goal |   99 cost |  10,759 actions | JumpingPuzzle('LLLLLLLLL.RRRRRRRRR', 'RR
    3,675 nodes |      736 goal |    7 cost |     742 actions | PancakeProblem((4, 6, 2, 5, 1, 3), (1, 2
   31,182 nodes |    5,198 goal |    8 cost |   5,205 actions | PancakeProblem((1, 3, 7, 5, 2, 6, 4), (1
   56,144 nodes |   17,808 goal |  173 cost |  17,967 actions | TOTAL

uniform_cost_search:
      948 nodes |      109 goal |    4 cost |     112 actions | PourProblem((1, 1, 1), 13)
    3,499 nodes |      389 goal |    9 cost |     397 actions | PourProblem((0, 0, 0), 21)
    1,696 nodes |      190 goal |   10 cost |     204 actions | GreenPourProblem((1, 1, 1), 13)
    4,072 nodes |      454 goal |   21 cost |     463 actions | GreenPourProblem((0, 0, 0), 21)
       82 nodes |      

KeyboardInterrupt: 

Esto confirma algunas de las cosas que ya sabíamos: A* y uniform-cost search son óptimos, pero los otros no lo son. A* explora menos nodos que uniform-cost.